In [1]:
from rdflib import Graph
from rdflib import URIRef
from rdflib.namespace import RDF
from rdflib import Namespace
import nltk
import codecs
import csv
from nltk.corpus import wordnet as wn
from num2words import num2words
from nltk.stem.snowball import PortugueseStemmer
from xml.dom import minidom
import string
import pickle
import numpy
import nltk
import re
import json
import codecs
from nltk.stem.snowball import PortugueseStemmer
from xml.dom import minidom
import string
import pickle
import numpy
import nltk
import re
import nltk
from nltk.corpus import wordnet as wn
from num2words import num2words
from gensim.models import KeyedVectors
from nltk.stem import WordNetLemmatizer
import pandas as pd

c:\program files\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
stemmer = PortugueseStemmer()
stopwords = nltk.corpus.stopwords.words('portuguese')
word_tokenize = nltk.word_tokenize
punct = string.punctuation

def expand_synonyms(tokens, it=1):
    """Docstring."""
    def recursion(tokens):
        new = []
        for token in tokens:
            try:
                sin = tep[delaf[token]]
                if len(sin) < 2:
                    new += sin
            except:
                pass
        return list(set(tokens + new))
    for i in range(it):
        tokens = recursion(tokens)
    return tokens

def word2embeddings(tokens):
    """Docstring."""
    return [embeddings[emb_vocab.index(t)]
            if t in emb_vocab else embeddings[0] for t in tokens]

def stem(tokens):
    """Docstring."""
    return [stemmer.stem(t) for t in tokens]

def filter_stopwords(tokens):
    """Docstring."""
    return [i.lower() for i in tokens if
            i.lower() not in stopwords and i not in punct]

def get_palavras_words(tree):
    """Docstring."""
    tree = tree.replace('\\n', '\n')
    return re.findall('\[(.*)\]', tree)

In [3]:
def find_synonyms(word, language, deep):
    synonyms = _find_synonyms(word, language, deep, 0)
    synonyms = list(set(synonyms))
    return synonyms

In [4]:
def _find_synonyms(word, language, deep, actual_deep):   
    synonyms = []
    if(deep == actual_deep):
        return [word]
    else:
        for ss in wn.synsets(word, lang=language):
            for lemma in ss.lemma_names(lang=language):
                synonyms.extend(_find_synonyms(lemma, language, deep, actual_deep + 1))
        return synonyms

In [5]:
def find_hypernyms(word, language, deep):
    hypernyms = _find_hypernyms(word, language, deep, 0)
    hypernyms = list(set(hypernyms))
    return hypernyms

In [6]:
def _find_hypernyms(word, language, deep, actual_deep):   
    hypernyms = []
    if(deep == actual_deep):
        return [word]
    else:
        for ss in wn.synsets(word, lang=language):
            for h in ss.hypernyms():
                for lemma in h.lemma_names(lang=language):
                    hypernyms.extend(_find_hypernyms(lemma, language, deep, actual_deep + 1))
        return hypernyms

In [7]:
def find_hyponyms(word, language, deep):
    hyponyms = _find_hyponyms(word, language, deep, 0)
    hyponyms = list(set(hyponyms))
    return hyponyms

In [8]:
def _find_hyponyms(word, language, deep, actual_deep):   
    hyponyms = []
    if(deep == actual_deep):
        return [word]
    else:
        for ss in wn.synsets(word, lang=language):
            for h in ss.hyponyms():
                for lemma in h.lemma_names(lang=language):
                    hyponyms.extend(_find_hyponyms(lemma, language, deep, actual_deep + 1))
        return hyponyms

In [ ]:
find_synonyms('cachorro', 'por', 2)

In [ ]:
find_hypernyms('cachorro', 'por', 2)

In [ ]:
find_hyponyms('cachorro', 'por', 2)

In [9]:
def replace_word_synonyms(sentence1, sentence2, language, synonym_deep = 2):
    sentence1 = sentence1.split()
    sentence2 = sentence2.split()
    
    sentence2_grams = ['_'.join(x) for x in nltk.ngrams(sentence2, 3)]
    
    lt = []
    
    for t in sentence1:
        if t in sentence2:
            lt.append(t)
        else:
            ths = find_synonyms(t, language, synonym_deep)
            ts = t
            for th in ths:
                if th in sentence2 or th in sentence2_grams:
                    ts = th
                    break
            lt.append(ts)
                    
    return ' '.join(lt)

In [10]:
def replace_word_hyponym(sentence1, sentence2, language, hyponym_deep = 2):
    sentence1 = sentence1.split()
    sentence2 = sentence2.split()
    
    sentence2_grams = ['_'.join(x) for x in nltk.ngrams(sentence2, 3)]
    
    lt = []
    
    for t in sentence1:
        if t in sentence2:
            lt.append(t)
        else:
            ths = find_hyponyms(t, language, hyponym_deep)
            ts = t
            for th in ths:
                if th in sentence2 or th in sentence2_grams:
                    ts = th
                    break
            lt.append(ts)
                    
    return ' '.join(lt)

In [11]:
def replace_word_hypernyms(sentence1, sentence2, language, hyperonym_deep = 2):
    sentence1 = sentence1.split()
    sentence2 = sentence2.split()
    
    sentence2_grams = ['_'.join(x) for x in nltk.ngrams(sentence2, 3)]
    
    lt = []
    
    for t in sentence1:
        if t in sentence2:
            lt.append(t)
        else:
            ths = find_hypernyms(t, language, hyperonym_deep)
            ts = t
            for th in ths:
                if th in sentence2 or th in sentence2_grams:
                    ts = th
                    break
            lt.append(ts)
                    
    return ' '.join(lt)

In [12]:
def replace_word_hypernyms_of_synonyms(sentence1, sentence2, language, synonym_deep = 2, hyperonym_deep = 2):   
    sentence1 = sentence1.split()
    sentence2 = sentence2.split()
    
    sentence2_grams = ['_'.join(x) for x in nltk.ngrams(sentence2, 3)]
    
    lt = []
    
    for t in sentence1:
        if t in sentence2:
            lt.append(t)
        else:
            reserved_words = find_synonyms(t, language, synonym_deep)
            reserved_words.append([find_hypernyms(x, language, hyperonym_deep) for x in reserved_words])
            
            changed_word = t
            for x in reserved_words:
                if x in sentence2 or x in sentence2_grams:
                    changed_word = x.replace('_', ' ')
                    
            lt.append(changed_word)
                    
    return ' '.join(lt)

In [ ]:
replace_word_synonyms('o jogo de futebol foi confirmar excelente hoje e um cachorro invadiu o campo', 'o inter afirmar jogou bem no campeonato, mas houve um cão invadiu o campo', 'por')

In [ ]:
replace_word_hypernyms_of_synonyms('o jogo de futebol foi confirmar excelente hoje e um cachorro invadiu o campo', 'o inter afirmar jogou bem no campeonato, mas houve um cão invadiu o campo', 'por')

In [13]:
def is_number_tryexcept(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [14]:
def number_to_word(word, language):

    if language == 'por':
        language = 'pt_BR'
    elif language == 'eng':
        language = 'en'

    try:
        return num2words(float(word), to = 'cardinal', lang = language)
    except NotImplementedError:
        return word

In [ ]:
number_to_word('2013', 'por')

In [15]:
def parseSentenceFile_synonyms(df, output, lang = 'por', to_json = False):
    data = []
    for index, row in df.iterrows():
        new = {}
        t1 = row['t']
        t2 = row['h']

        # Remove stopwords and tokenize
        t1 = filter_stopwords(nltk.word_tokenize(t1))
        t2 = filter_stopwords(nltk.word_tokenize(t2))
        
        # Remove spaces and to lower
        t1 = [t.lower().strip() for t in t1]
        t2 = [t.lower().strip() for t in t2]

        # Convert numbers to words
        t1 = [number_to_word(s.strip(), lang).replace('vírgula zero', '') if is_number_tryexcept(s.strip()) else s for s in t1]
        t2 = [number_to_word(s.strip(), lang).replace('vírgula zero', '') if is_number_tryexcept(s.strip()) else s for s in t2]

        t1 = ' '.join(t1)
        t2 = ' '.join(t2)
               
        # Replace hyperonyms
        # pth1 = [k if k in nltk.word_tokenize(t2) else t for k in find_hypernyms(t, lang) if t in nltk.word_tokenize(t2) else t for t in nltk.word_tokenize(t1)]
        # pth2 = [k if k in nltk.word_tokenize(t1) else t for k in find_hypernyms(t, lang) if t in nltk.word_tokenize(t1) else t for t in nltk.word_tokenize(t2)]
        pth1 = replace_word_synonyms(t1, t2, lang, 3)
        pth2 = replace_word_synonyms(t2, t1, lang, 3)

        new['t'] = pth1
        new['h'] = pth2
        
        # Similaridade apenas para conjunto de treinamento
        try:
            new['entailment'] = row['entailment']
            new['similarity'] = row['similarity']
        except:
            pass
        data.append(new)

    with codecs.open(output, "wb", encoding='utf8', errors='ignore') as fp:
        if to_json == True:
            json.dump(data, fp, ensure_ascii=False)
        else:
            pickle.dump(data, fp)

In [16]:
def parseSentenceFile_hyperonyms(df, output, lang = 'por', to_json = False):
    data = []
    for index, row in df.iterrows():
        new = {}
        t1 = row['t']
        t2 = row['h']

        # Remove stopwords and tokenize
        t1 = filter_stopwords(nltk.word_tokenize(t1))
        t2 = filter_stopwords(nltk.word_tokenize(t2))
        
        # Remove spaces and to lower
        t1 = [t.lower().strip() for t in t1]
        t2 = [t.lower().strip() for t in t2]

        # Convert numbers to words
        t1 = [number_to_word(s.strip(), lang).replace('vírgula zero', '') if is_number_tryexcept(s.strip()) else s for s in t1]
        t2 = [number_to_word(s.strip(), lang).replace('vírgula zero', '') if is_number_tryexcept(s.strip()) else s for s in t2]
               
        # Replace hyperonyms
        # pth1 = [k if k in nltk.word_tokenize(t2) else t for k in find_hypernyms(t, lang) if t in nltk.word_tokenize(t2) else t for t in nltk.word_tokenize(t1)]
        # pth2 = [k if k in nltk.word_tokenize(t1) else t for k in find_hypernyms(t, lang) if t in nltk.word_tokenize(t1) else t for t in nltk.word_tokenize(t2)]
        pth1 = replace_word_hypernyms(t1, t2, lang, 3)
        pth2 = replace_word_hypernyms(t2, t1, lang, 3)

        new['t'] = pth1
        new['h'] = pth2

        # Similaridade apenas para conjunto de treinamento
        try:
            new['entailment'] = row['entailment']
            new['similarity'] = row['similarity']
        except:
            pass
        data.append(new)

    with codecs.open(output, "wb", encoding='utf8', errors='ignore') as fp:
        if to_json == True:
            json.dump(data, fp, ensure_ascii=False)
        else:
            pickle.dump(data, fp)

In [17]:
def parseSentenceFile_synonyms_hyperonyms(df, output, lang = 'por', to_json = False):
    data = []
    for index, row in df.iterrows():
        
        new = {}
        t1 = row['t']
        t2 = row['h']

        
        # Remove stopwords and tokenize
        t1 = filter_stopwords(nltk.word_tokenize(t1))
        t2 = filter_stopwords(nltk.word_tokenize(t2))
        
        # Remove spaces and to lower
        t1 = [t.lower().strip() for t in t1]
        t2 = [t.lower().strip() for t in t2]

        # Convert numbers to words
        t1 = [number_to_word(s.strip(), lang).replace('vírgula zero', '') if is_number_tryexcept(s.strip()) else s for s in t1]
        t2 = [number_to_word(s.strip(), lang).replace('vírgula zero', '') if is_number_tryexcept(s.strip()) else s for s in t2]

        t1 = ' '.join(t1)
        t2 = ' '.join(t2)
        
        
        pts1 = replace_word_hypernyms_of_synonyms(t1, t2, lang, 3)
        pts2 = replace_word_hypernyms_of_synonyms(t2, t1, lang, 3)
              
        new['t'] = pts1
        new['h'] = pts2

        # Similaridade apenas para conjunto de treinamento
        try:
            new['entailment'] = row['entailment']
            new['similarity'] = row['similarity']
        except:
            pass
        
        data.append(new)

    with codecs.open(output, "wb", encoding='utf8', errors='ignore') as fp:
        if to_json == True:
            json.dump(data, fp, ensure_ascii=False)
        else:
            pickle.dump(data, fp)

In [18]:
def read_xml_propor_dataset(input):
    df = pd.DataFrame(columns=['t', 'h', 'entailment', 'similarity'])
    index = 0
    xml = minidom.parse(input)
    pairs = xml.getElementsByTagName('pair')
    data = []
    for i in pairs:
        t = i.getElementsByTagName('t')[0].childNodes[0].data.strip()
        h = i.getElementsByTagName('h')[0].childNodes[0].data.strip()
        ent = i.attributes['entailment'].value
        sim = i.attributes['similarity'].value
        
        index = index + 1
        df.loc[index] = [t, h, ent, sim]
    return df

In [19]:
def read_json_propor_dataset(input):
    with codecs.open(input, 'rb', encoding='utf8') as fr:
        df_json = json.load(fr)
    
    index = 0
    df = pd.DataFrame(columns=['t', 'h', 'entailment', 'similarity'])
    for x in df_json:
        if isinstance(x, dict):
            x = list(x.values())
            
        t = x[0]
        h = x[1]
        ent = x[2]
        sim = x[3]
        
        index = index + 1
        df.loc[index] = [t, h, ent, sim]
    
    return df

** Portuguese sentences parsing **

In [ ]:
parseSentenceFile_synonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setP1.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setP1-synonym.json',
         'por',
         True)

In [ ]:
parseSentenceFile_synonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setP2.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setP2-synonym.json',
         'por',
         True)

In [ ]:
parseSentenceFile_synonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setP3.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setP3-synonym.json',
         'por',
         True)

In [ ]:
parseSentenceFile_synonyms_hyperonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setP1.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setP1-synonym-hyperonym.json',
         'por',
         True)

In [ ]:
parseSentenceFile_synonyms_hyperonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setP2.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setP2-synonym-hyperonym.json',
         'por',
         True)

In [ ]:
parseSentenceFile_synonyms_hyperonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setP3.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setP3-synonym-hyperonym.json',
         'por',
         True)

** English sentences parsing **

In [ ]:
parseSentenceFile_synonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setE1.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setE1-synonym.json',
         'eng',
         True)

In [ ]:
parseSentenceFile_synonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setE2.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setE2-synonym.json',
         'eng',
         True)

In [ ]:
parseSentenceFile_synonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setE3.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setE3-synonym.json',
         'eng',
         True)

In [ ]:
parseSentenceFile_synonyms_hyperonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setE1.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setE1-synonym-hyperonym.json',
         'eng',
         True)

In [ ]:
parseSentenceFile_synonyms_hyperonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setE2.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setE2-synonym-hyperonym.json',
         'eng',
         True)

In [ ]:
parseSentenceFile_synonyms_hyperonyms(
        read_json_propor_dataset('C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/setE3.json'),
         'C:/Users/Asilva/Documents/Git/unists-python/Data/splitted/linguistic/setE3-synonym-hyperonym.json',
         'eng',
         True)

In [ ]:
1

In [ ]:
# embeddings = KeyedVectors.load_word2vec_format('C:/Users/Asilva/Documents/Git/unists-python/Data/glove_s1000/glove_s1000.txt', unicode_errors="ignore")

In [ ]:
from nltk import RSLPStemmer 

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
wordnet_lemmatizer.lemmatize('fiz', pos='v', )

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('C:/Users/Asilva/Desktop/dados-cursos.csv', sep=";", low_memory=False)

In [ ]:
with codecs.open('C:/Users/Asilva/Desktop/dados-cursos.json', 'w', encoding='utf8') as filew:
    filew.write(df.to_json(orient='records', lines=True))

In [ ]:
df.to_csv('C:/Users/Asilva/Desktop/dados-cursos-readable.csv', sep=";",quotechar='"',index=False,
            quoting=csv.QUOTE_ALL, encoding="utf-8")